<a href="https://colab.research.google.com/github/clairecleverlamb/smart-chatbots/blob/main/Fitness%26Nutrition_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio transformers torch numpy requests matplotlib


In [ ]:
import gradio as gr
import numpy as np
import requests
import random  # You missed this import for random.choice
import matplotlib.pyplot as plt
from transformers import pipeline

# Load NLP Model
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Intent categories
CANDIDATE_LABELS = ["greeting", "workout_request", "meal_plan_request", "motivation", "set_goal", "track_progress", "calorie_calculation", "out_of_scope"]

# User state
user_state = {"goal": None, "diet": None, "calories": None, "progress": [], "workouts": [], "meals": []}


Device set to use cpu


In [ ]:
def fetch_random_workout():
    """Fetch a random workout from Wger API."""
    url = "https://wger.de/api/v2/exercise/?limit=10"  # Get 10 workouts

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Print the raw data for debugging
        print("API Response:", data)

        if "results" in data and len(data["results"]) > 0:
            workout = random.choice(data["results"])

            # Get workout details from the response
            workout_id = workout.get("id", "Unknown ID")
            category = workout.get("category", "Unknown Category")
            equipment = workout.get("equipment", "No equipment info")

            # Use a placeholder description if not available
            description = workout.get("description", "No description available.")

            # Fallback description based on category and equipment
            if description == "No description available.":
                description = f"Category: {category}, Equipment: {', '.join(map(str, equipment)) if equipment else 'None'}"

            return f"🏋️ Try this workout (ID: {workout_id}, Category: {category}):\n\n📖 {description}"
        else:
            return "No workouts available right now."

    except requests.exceptions.RequestException as e:
        return f"❌ Error fetching workout: {str(e)}"


In [ ]:
def fetch_meal_plan():
    """Fetch a meal plan from an API."""
    api_key = "a461777ee5f844c8af95db4f81bbb613"
    url = f"https://api.spoonacular.com/mealplanner/generate?apiKey={api_key}"

    response = requests.get(url)

    if response.status_code == 200:
        # Parsing the JSON data from the API response
        data = response.json()

        # Extracting the first meal's title (you can modify this based on your needs)
        return data["meals"][0]["title"]

    return "Grilled chicken with vegetables."

In [ ]:

def fitness_chat(user_input: str, state: dict):
    if state is None:
        state = {"goal": None, "diet": None, "calories": None, "progress": [], "workouts": [], "meals": []}

    text = user_input.lower()
    result = classifier(text, CANDIDATE_LABELS)
    top_label, top_score = result["labels"][0], result["scores"][0]

    # Print the classifier results for debugging
    print(f"Classifier Result: {result}")
    print(f"Top Label: {top_label}, Top Score: {top_score}")

    if top_score < 0.7 or top_label == "out_of_scope":
        user_message = {"role": "user", "content": user_input}
        bot_message = {"role": "assistant", "content": "I’m not sure about that. Can you rephrase?"}
        return [user_message, bot_message], state

    if top_label == "greeting":
        user_message = {"role": "user", "content": user_input}
        bot_message = {"role": "assistant", "content": "Hi! Are you looking for workouts, meal plans, or calorie info?"}
        return [user_message, bot_message], state

    elif top_label == "workout_request":
        workout = fetch_random_workout()
        state["workouts"].append(workout)
        user_message = {"role": "user", "content": user_input}
        bot_message = {"role": "assistant", "content": f"Here's a workout suggestion: {workout}"}
        return [user_message, bot_message], state

    elif top_label == "meal_plan_request":
        meal = fetch_meal_plan()
        state["meals"].append(meal)
        user_message = {"role": "user", "content": user_input}
        bot_message = {"role": "assistant", "content": f"Try this meal: {meal}"}
        return [user_message, bot_message], state

    elif top_label == "motivation":
        user_message = {"role": "user", "content": user_input}
        bot_message = {"role": "assistant", "content": "You're doing great! Stay consistent and trust the process. 💪"}
        return [user_message, bot_message], state

    elif top_label == "set_goal":
        state["goal"] = text
        user_message = {"role": "user", "content": user_input}
        bot_message = {"role": "assistant", "content": f"Goal set! You’re aiming for: {text}. Keep it up!"}
        return [user_message, bot_message], state

    elif top_label == "track_progress":
        state["progress"].append(text)
        user_message = {"role": "user", "content": user_input}
        bot_message = {"role": "assistant", "content": f"Progress logged: {text}. Keep pushing! 🚀"}
        return [user_message, bot_message], state

    elif top_label == "calorie_calculation":
        user_message = {"role": "user", "content": user_input}
        bot_message = {"role": "assistant", "content": "Please provide your weight (kg), height (cm), and age."}
        return [user_message, bot_message], state

    # Default response if no specific label matches
    user_message = {"role": "user", "content": user_input}
    bot_message = {"role": "assistant", "content": "I'm here to help! Ask me about workouts, meals, or tracking progress."}
    return [user_message, bot_message], state


In [ ]:
def calculate_calories(user_input, state):
    try:
        weight, height, age = map(float, user_input.split())
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
        daily_calories = bmr * 1.55
        state["calories"] = daily_calories
        return f"Your estimated daily caloric need is {int(daily_calories)} kcal.", state
    except:
        return "Enter weight (kg), height (cm), and age (e.g., '70 175 25').", state

In [ ]:
def plot_progress(state):
    labels = ["Workouts", "Meals Logged", "Progress Updates"]
    values = [len(state["workouts"]), len(state["meals"]), len(state["progress"])]

    fig, ax = plt.subplots()
    ax.bar(labels, values, color=['blue', 'green', 'orange'])
    ax.set_title("Your Fitness Progress")
    ax.set_ylabel("Entries Logged")
    return fig


In [ ]:
# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# 🏋️‍♂️ Fitness Chatbot + Progress Dashboard")

    chatbot = gr.Chatbot(type="messages")
    state = gr.State({"goal": None, "diet": None, "calories": None, "progress": [], "workouts": [], "meals": []})

    txt = gr.Textbox(show_label=False, placeholder="Type or speak your question.")

    # Fix for gr.Audio (remove 'source' parameter)
    mic = gr.Audio(sources=["microphone"], type="numpy")

    btn_dashboard = gr.Button("Show Progress")

    txt.submit(fitness_chat, inputs=[txt, state], outputs=[chatbot, state])
    mic.change(fitness_chat, inputs=[mic, state], outputs=[chatbot, state])

    # Plot component for progress
    plot = gr.Plot()  # Define a plot component
    btn_dashboard.click(plot_progress, inputs=[state], outputs=plot)

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3c576bccc11c6ac21f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
